In [64]:
import os
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
import math

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5, augment=True):
        self.data = []
        self.sequence_length = sequence_length
        self.num_classes = (105 - 30) // 10 + 1  # Speed classes from 30-39, ..., 100-105
        self.augment = augment  # Enable or disable augmentation
        self.preprocess_data(directory)
        self.normalize_features()

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                speed_class = 0 if speed < 30 else (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        points = line.strip().split(',')
                        if len(points) == 6:
                            frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if len(points) == 7:
                            frame,class_id, track_id, x1, y1, x2, y2 = map(float, line.strip().split(','))
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1, y1, x2, y2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = self.extract_features(frames)
                            overlap = 5  # Overlap for sequence extraction
                            for start_idx in range(0, len(features) - self.sequence_length + 1, self.sequence_length - overlap):
                                end_idx = start_idx + self.sequence_length
                                sequence = features[start_idx:end_idx]
                                self.data.append((sequence, speed_class))

    def extract_features(self, frames):
        features = []
        for i in range(1, len(frames)):
            current_frame = frames[i]
            previous_frame = frames[i-1]
            features.append(self.compute_frame_features(current_frame, previous_frame))
        return features

    def compute_frame_features(self, current_frame, previous_frame):
        x1, y1, x2, y2 = current_frame
        px1, py1, px2, py2 = previous_frame

        width, height = x2 - x1, y2 - y1
        p_width, p_height = px2 - px1, py2 - py1

        x2_change = x2 - px2
        y2_change = y2 - py2
        x1_change = x1 - px1
        y1_change = y1 - py1

        width_change = width - p_width
        height_change = height - p_height
        area_change = (width * height) - (p_width * p_height)
        perimeter_change = (2 * (width + height)) - (2 * (p_width + p_height))

        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        p_center_x, p_center_y = (px1 + px2) / 2, (py1 + py2) / 2
        center_x_change = center_x - p_center_x
        center_y_change = center_y - p_center_y
        distance_moved = math.sqrt(center_x_change ** 2 + center_y_change ** 2)

        velocity = distance_moved/0.03  # Assuming constant frame rate
        p_velocity = (math.sqrt((px2 - px1) ** 2 + (py2 - py1) ** 2))/0.03
        acceleration = abs(velocity - p_velocity)

        feature_vector = [x1_change, y1_change, x2_change, y2_change,
                          center_x_change, center_y_change, distance_moved, velocity, acceleration]
        return feature_vector

    def normalize_features(self):
        all_features = [feature for sequence, _ in self.data for feature in sequence]
        all_features = np.array(all_features)
        self.mean = np.mean(all_features, axis=0)
        self.std = np.std(all_features, axis=0)
        for i, (sequence, speed_class) in enumerate(self.data):
            normalized_sequence = (sequence - self.mean) / (self.std)
            self.data[i] = (normalized_sequence, speed_class)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        if self.augment:
            inputs = self.apply_augmentation(inputs)
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)

    def apply_augmentation(self, inputs):
        if random.random() > 0.5:
            inputs = self.add_noise(inputs)
        if random.random() > 0.5:
            inputs = self.time_shift(inputs, shift=random.choice([-1, 1]))
        if random.random() > 0.5:
            inputs = self.scale_features(inputs, scale=random.uniform(0.9, 1.1))
        if random.random() > 0.5:
            inputs = self.mirror_features(inputs)
        return inputs

    def add_noise(self, features, noise_level=0.05):
        noise = np.random.normal(0, noise_level, features.shape)
        return features + noise

    def time_shift(self, features, shift=1):
        if shift > 0:
            return np.vstack([np.zeros((shift, features.shape[1])), features[:-shift]])
        elif shift < 0:
            return np.vstack([features[-shift:], np.zeros((-shift, features.shape[1]))])
        return features

    def scale_features(self, features, scale=1.1):
        return features * scale

    def mirror_features(self, features):
        features_copy = features.copy()
        features_copy[:, [0, 2]] = -features_copy[:, [0, 2]]  # Assume these indices are the x-coordinates
        return features_copy


In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        weights = F.softmax(self.linear(x), dim=1)
        weighted_output = weights * x
        return weighted_output.sum(1), weights

class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=feature_size, out_channels=embedding_dim, kernel_size=1)
        self.lstm = nn.RNN(embedding_dim, hidden_dim, batch_first=True, num_layers=4)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(32, output_size)  
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=128, kernel_size=1)
        self.attention = Attention(hidden_dim)
        self.fc1 = nn.Linear(128, 32)

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.conv1(x)
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x, attn_weights = self.attention(x)
        x = x.unsqueeze(2)
        x = self.conv2(x)
        x = x.squeeze(2)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  
def init_weights(m):
    if type(m) == nn.LSTM:
        torch.nn.init.xavier_uniform_(m.weight_ih_l0)
        torch.nn.init.orthogonal_(m.weight_hh_l0)
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)

In [86]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [87]:

def train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs):
    for epoch in range(epochs):
        model.train()
        Train_total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # First, update the parameters with the current learning rate
            
            Train_total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)
        
        scheduler.step()  # After optimizer updates, adjust the learning rate

        if epoch % 10 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            total_loss = 0
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)
                    
            train_accuracy = 100 * correct_train / total_train
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {Train_total_loss / len(train_loader)} Test Loss: {total_loss / len(test_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')


In [88]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR


train_dataset = SpeedDataset('outputs', sequence_length=10,augment=True)
test_dataset = SpeedDataset('test_outputs', sequence_length=10,augment=False)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
max_classes = train_dataset.num_classes
model = SpeedPredictor(sequence_length=10, feature_size=9, embedding_dim=64, hidden_dim=128, output_size=max_classes)
model.to(device)  
model.apply(init_weights)
criterion = nn.CrossEntropyLoss().to(device)  
optimizer = SGD(model.parameters(), lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.9999) 
train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=400)


7822
Epoch 1: Train Loss: 2.000758582718518 Test Loss: 1.9770356006920338, Train Accuracy: 18.38%, Test Accuracy: 20.09%
Epoch 11: Train Loss: 1.32905147927148 Test Loss: 1.217785923043266, Train Accuracy: 44.21%, Test Accuracy: 41.46%
Epoch 21: Train Loss: 1.209869509083884 Test Loss: 1.3280291613191366, Train Accuracy: 48.45%, Test Accuracy: 41.93%
Epoch 31: Train Loss: 1.157703031569111 Test Loss: 1.050881806469988, Train Accuracy: 51.09%, Test Accuracy: 54.11%
Epoch 41: Train Loss: 1.1443912082788896 Test Loss: 1.0045670932158828, Train Accuracy: 50.29%, Test Accuracy: 56.33%
Epoch 51: Train Loss: 1.1303456106964422 Test Loss: 1.0982911344617605, Train Accuracy: 50.81%, Test Accuracy: 51.42%
Epoch 61: Train Loss: 1.1330472196851458 Test Loss: 1.3703800259681884, Train Accuracy: 51.34%, Test Accuracy: 38.13%
Epoch 71: Train Loss: 1.0964496125980299 Test Loss: 1.042203658435028, Train Accuracy: 52.30%, Test Accuracy: 54.59%
Epoch 81: Train Loss: 1.1031049039899086 Test Loss: 1.052538

In [89]:
from sklearn.metrics import confusion_matrix, classification_report
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [90]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[838 293  44   6   6   1   0   0]
 [ 94 931 315  68  53   3   5   0]
 [  4 122 738 240 153  21  12   0]
 [  1  19 141 620 258  66  49   0]
 [  0   1  28  63 571 289  85   0]
 [  2   0   5  17 100 371 230   8]
 [  0   0   1   8  36  70 516  66]
 [  0   0   1   1   6  17  99 130]]
               precision    recall  f1-score   support

  Class 30-39       0.89      0.71      0.79      1188
  Class 40-49       0.68      0.63      0.66      1469
  Class 50-59       0.58      0.57      0.58      1290
  Class 60-69       0.61      0.54      0.57      1154
  Class 70-79       0.48      0.55      0.51      1037
  Class 80-89       0.44      0.51      0.47       733
  Class 90-99       0.52      0.74      0.61       697
Class 100-109       0.64      0.51      0.57       254

     accuracy                           0.60      7822
    macro avg       0.61      0.59      0.59      7822
 weighted avg       0.62      0.60      0.61      7822



60.278701099463056

In [91]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[70 20  5  1  0  0  0  0]
 [ 5 56 48  5  8  0  0  0]
 [ 0 11 57 17  7  1  3  0]
 [ 0  0 25 49 35  8  6  0]
 [ 0  0  4  8 44 13  1  0]
 [ 0  1  1  1 14 23 38  2]
 [ 0  0  0  0  0  0  7  6]
 [ 0  0  0  0  0  2 16 14]]
               precision    recall  f1-score   support

  Class 30-39       0.93      0.73      0.82        96
  Class 40-49       0.64      0.46      0.53       122
  Class 50-59       0.41      0.59      0.48        96
  Class 60-69       0.60      0.40      0.48       123
  Class 70-79       0.41      0.63      0.49        70
  Class 80-89       0.49      0.29      0.36        80
  Class 90-99       0.10      0.54      0.17        13
Class 100-109       0.64      0.44      0.52        32

     accuracy                           0.51       632
    macro avg       0.53      0.51      0.48       632
 weighted avg       0.59      0.51      0.52       632



50.63291139240506